# **Improving Time Series Classification Accuracy Using Self-Supervised Learning.**


# **Import data and preprocessing**

In [ ]:
import zipfile
import torch
import os

# Step 1: Extract files from Gesture.zip
with zipfile.ZipFile('/content/Gesture.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Gesture_data')

# Step 2: Load each extracted .pt file and access 'samples' and 'labels'
train_data = torch.load('/content/Gesture_data/train.pt')
val_data = torch.load('/content/Gesture_data/val.pt')
test_data = torch.load('/content/Gesture_data/test.pt')

# Extract data and labels using 'samples' and 'labels' keys
X_train = train_data['samples']
y_train = train_data['labels']
X_val = val_data['samples']
y_val = val_data['labels']
X_test = test_data['samples']
y_test = test_data['labels']

# Check the shapes of tensors
print("Train Data Shape:", X_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Labels Shape:", y_val.shape)
print("Test Data Shape:", X_test.shape)
print("Test Labels Shape:", y_test.shape)



Train Data Shape: torch.Size([320, 3, 206])
Train Labels Shape: torch.Size([320])
Validation Data Shape: torch.Size([120, 3, 206])
Validation Labels Shape: torch.Size([120])
Test Data Shape: torch.Size([120, 3, 206])
Test Labels Shape: torch.Size([120])


<ipython-input-4-657871ab67cf>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load('/content/Gesture_data/train.pt')
<ipython-input-4-657871ab67cf>:11:

# **Self-Supervised Learning Model(e.g., Autoencoder)**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize model, loss function, and optimizer
input_dim = X_train.shape[-1]
autoencoder = Autoencoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)


# **Train the Self-Supervised Model**

In [ ]:
# Ensure input tensors are in float32 format
X_train = X_train.float()
X_val = X_val.float()
X_test = X_test.float()

# Training the autoencoder
num_epochs = 20
batch_size = 32
train_loader = torch.utils.data.DataLoader(X_train, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for data in train_loader:
        # Convert data to float32 if necessary
        data = data.float()

        # Forward pass
        output = autoencoder(data)
        loss = criterion(output, data)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/20], Loss: 1.1144
Epoch [2/20], Loss: 0.9769
Epoch [3/20], Loss: 0.7050
Epoch [4/20], Loss: 0.6698
Epoch [5/20], Loss: 0.5889
Epoch [6/20], Loss: 0.5780
Epoch [7/20], Loss: 0.5352
Epoch [8/20], Loss: 0.5281
Epoch [9/20], Loss: 0.5699
Epoch [10/20], Loss: 0.5305
Epoch [11/20], Loss: 0.5328
Epoch [12/20], Loss: 0.5567
Epoch [13/20], Loss: 0.4820
Epoch [14/20], Loss: 0.5041
Epoch [15/20], Loss: 0.4797
Epoch [16/20], Loss: 0.4494
Epoch [17/20], Loss: 0.5402
Epoch [18/20], Loss: 0.5741
Epoch [19/20], Loss: 0.5199
Epoch [20/20], Loss: 0.5724


# **Extract Features for Classification**

In [ ]:
# Set the model to evaluation mode
autoencoder.eval()

# Extract features for each dataset using the encoder part of the autoencoder
with torch.no_grad():
    X_train_features = autoencoder.encoder(X_train).view(X_train.size(0), -1)
    X_val_features = autoencoder.encoder(X_val).view(X_val.size(0), -1)
    X_test_features = autoencoder.encoder(X_test).view(X_test.size(0), -1)

print("Train Features Shape:", X_train_features.shape)
print("Validation Features Shape:", X_val_features.shape)
print("Test Features Shape:", X_test_features.shape)


Train Features Shape: torch.Size([320, 6592])
Validation Features Shape: torch.Size([120, 6592])
Test Features Shape: torch.Size([120, 6592])


# **Train a Classifier Using Extracted Features**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Convert features and labels to numpy arrays for use with sklearn
X_train_features_np = X_train_features.cpu().numpy()
y_train_np = y_train.cpu().numpy()
X_val_features_np = X_val_features.cpu().numpy()
y_val_np = y_val.cpu().numpy()
X_test_features_np = X_test_features.cpu().numpy()
y_test_np = y_test.cpu().numpy()

# Train a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_features_np, y_train_np)

# Evaluate the classifier on the test set
y_pred = classifier.predict(X_test_features_np)
accuracy = accuracy_score(y_test_np, y_pred)
print("Classification Accuracy on Test Set:", accuracy)


Classification Accuracy on Test Set: 0.6583333333333333


# **Evaluate Performance**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate a detailed classification report
print("Classification Report:\n", classification_report(y_test_np, y_pred))

# Display the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test_np, y_pred))


Classification Report:
               precision    recall  f1-score   support

         0.0       0.73      0.73      0.73        15
         1.0       0.80      0.80      0.80        15
         2.0       0.87      0.87      0.87        15
         3.0       0.62      0.67      0.65        15
         4.0       0.40      0.13      0.20        15
         5.0       0.17      0.20      0.18        15
         6.0       0.78      0.93      0.85        15
         7.0       0.78      0.93      0.85        15

    accuracy                           0.66       120
   macro avg       0.64      0.66      0.64       120
weighted avg       0.64      0.66      0.64       120

Confusion Matrix:
 [[11  1  0  0  0  2  0  1]
 [ 0 12  0  0  1  0  2  0]
 [ 0  0 13  0  0  2  0  0]
 [ 1  0  0 10  0  3  0  1]
 [ 3  0  0  3  2  7  0  0]
 [ 0  2  2  3  2  3  1  2]
 [ 0  0  0  0  0  1 14  0]
 [ 0  0  0  0  0  0  1 14]]
